In [ ]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    get_load_config_from_yaml,
)
from importlib import reload
import core
import keras
import core.keras_models.RegressionTransformer as RegressionTransformer
import core.utils as utils

PLOTS_DIR = f"plots/regression_transformer/"
MODEL_DIR = f"models/regression_transformer/"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


data_config = DataProcessor.load_from_npz(
    load_config.data_path["nominal"], max_events=4_000_000, event_numbers="even"
)

X_train, y_train = DataProcessor.get_data()
del DataProcessor  # Free memory

In [ ]:
reload(RegressionTransformer)
Transformer = RegressionTransformer.FeatureConcatTransformerReconstructor(data_config, name="Transformer")

In [ ]:
Transformer.build_model(
    hidden_dim=64,
    num_layers=6,
    dropout_rate=0.1,
)


In [ ]:
Transformer.adapt_normalization_layers(X_train)
Transformer.compile_model(
    loss={
        "assignment": utils.AssignmentLoss(),
        "normalized_regression": utils.PtEtaPhiLoss(),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [utils.AssignmentAccuracy(name="accuracy")],
        "normalized_regression": [utils.RegressionDeviation(name="deviation")],
    },
    #add_physics_informed_loss=True,
    loss_weights={"assignment": 1.0, "normalized_regression": 1.0}
)

In [ ]:
Transformer.train_model(
    epochs=10,
    data=X_train,
    sample_weights=utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            verbose=1,
            mode="min",
            min_lr=1e-6,
        ),
        keras.callbacks.TerminateOnNaN(),
    ],
    validation_split=0.1,
)

In [ ]:
Transformer.save_model(MODEL_DIR + "odd_model.keras")

In [ ]:
upscale_layer = Transformer.model.get_layer("regression")

In [ ]:
import numpy as np
import tensorflow as tf
np.abs(upscale_layer(y_train["regression"] / y_train["regression"].std(axis = 0)) - X_train["regression"]).mean()

In [ ]:
from core.components import reco_W_mass_deviation
leptons = X_train["lep_inputs"]
neutrino_pred = Transformer.reconstruct_neutrinos(X_train)
neutrino_true = X_train["regression"]
neutrino_nu_flows = X_train["nu_flows_neutrino_truth"]

In [ ]:
reco_W_mass_deviation(tf.convert_to_tensor(neutrino_pred, dtype=tf.float32), tf.convert_to_tensor(leptons, dtype=tf.float32)).numpy().mean()

In [ ]:
reco_W_mass_deviation(tf.convert_to_tensor(neutrino_true, dtype=tf.float32), tf.convert_to_tensor(leptons, dtype=tf.float32)).numpy().mean()

In [ ]:
reco_W_mass_deviation(tf.convert_to_tensor(neutrino_nu_flows, dtype=tf.float32), tf.convert_to_tensor(leptons, dtype=tf.float32)).numpy().mean()